In [1]:
import datetime as dt
import time
import requests
import random
import numpy as np
import pandas as pd
from config import weather_api_key


In [2]:
# Other local versions of this global variable will be lats, lat, lts, parallel
latitude = np.random.uniform(-90,90, size=1500)
latitude

array([ 47.85120475, -72.38823096,  38.69037412, ...,  18.09266339,
        60.52161987,   5.96235474])

In [3]:
# Other local versions of this global variable will be long, lon, lng, meridian, 
longitude = np.random.uniform(-180,180, size=1500)
longitude

array([-153.50492994,  -26.99672584, -143.78246577, ..., -117.37456311,
        138.88833257, -117.61184366])

In [4]:
lat_long = list(zip(latitude, longitude))
lat_long[0]

(47.85120474566784, -153.504929935395)

In [5]:
from citipy import citipy
cities = set()
for lat, long in lat_long:
    c = citipy.nearest_city(lat, long)
    if c not in cities:
        cities.add(c)#, c.country_code))
cities = list(cities)
cities[0]

In [6]:
params= {'APPID':weather_api_key,
        'q': 'Boston',
        'units':'Imperial'}

url = f"https://api.openweathermap.org/data/2.5/weather"#?lat={lat_long[0][0]}&lon={lat_long[0][1]}&appid={weather_api_key}"
response = requests.get(url, params).json()
response['cod']

200

In [7]:
# pd.DataFrame has: City, Country, Date, lat, lon, Max Temp, Humidity, Cloudiness, Wind Speed
city_dict = []

# use the cities list to start the dataframe
for place in cities:
    p = dict()
    p['City'] = place.city_name.capitalize()
    p['Country'] = place.country_code.upper()
    city_dict.append(p)    
    
city_df = pd.DataFrame(city_dict)
city_df['Date'] = ''
city_df['Lat'] = ''
city_df['Lon'] = ''
city_df['Max Temp'] = ''
city_df['Humidity'] = ''
city_df['Cloudiness'] = ''
city_df['Wind Speed'] = ''
city_df['Description'] = ''
city_df


,City,Country,Date,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,Husavik,IS,,,,,,,,
1,Taltal,CL,,,,,,,,
2,Barcelos,BR,,,,,,,,
3,Sulangan,PH,,,,,,,,
4,Hvammstangi,IS,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...
615,Ancud,CL,,,,,,,,
616,Ambon,ID,,,,,,,,
617,Ambulu,ID,,,,,,,,
618,Tarakan,ID,,,,,,,,


In [8]:
# retieve the weather information for each city and fill in the data
# Note: Open Weather API only allows 60 api calls per minute
for index in city_df.index:
    if (index % 50 == 0) and (index > 0):
        print(index, 'Sleep Here')
        time.sleep(60)
    else: 
        pass
# # Retrive the weather information for each city
    params['q'] = city_df['City'][index]
    response = requests.get(url, params).json()

    # Save the weather information for the city    
    try:
        city_df['Date'][index] = dt.datetime.utcfromtimestamp(response['dt']).strftime('%Y-%m-%d %H:%M:%S') 
        city_df['Lat'][index] = response['coord']['lat']
        city_df['Lon'][index] = response['coord']['lon']
        city_df['Max Temp'][index] = response['main']['temp_max']
        city_df['Humidity'][index] = response['main']['humidity']
        city_df['Wind Speed'][index] = response['wind']['speed']
        city_df['Description'][index] = response['weather'][0]['description']
        city_df['Cloudines'][index] = response['clouds']['all']
    except KeyError:
        print('City:', params['q'], '  |  KeyError, HTTP Code:', response['cod'])
    

city_df

City: Husavik   |  KeyError, HTTP Code: 200
City: Taltal   |  KeyError, HTTP Code: 200
City: Barcelos   |  KeyError, HTTP Code: 200
City: Sulangan   |  KeyError, HTTP Code: 200
City: Hvammstangi   |  KeyError, HTTP Code: 404
City: Pochutla   |  KeyError, HTTP Code: 200
City: Bengkulu   |  KeyError, HTTP Code: 200
City: Zhigalovo   |  KeyError, HTTP Code: 200
City: Zhigansk   |  KeyError, HTTP Code: 200
City: Gornyy   |  KeyError, HTTP Code: 200
City: Pingliang   |  KeyError, HTTP Code: 200
City: Biak   |  KeyError, HTTP Code: 200
City: Warwick   |  KeyError, HTTP Code: 200
City: Saint-philippe   |  KeyError, HTTP Code: 200
City: Dwarka   |  KeyError, HTTP Code: 200
City: Koygorodok   |  KeyError, HTTP Code: 200
City: Weiser   |  KeyError, HTTP Code: 200
City: Bikin   |  KeyError, HTTP Code: 200
City: Cabo san lucas   |  KeyError, HTTP Code: 200
City: Nong kung si   |  KeyError, HTTP Code: 200
City: San ramon de la nueva oran   |  KeyError, HTTP Code: 200
City: East london   |  KeyError

KeyboardInterrupt: 